<a href="https://colab.research.google.com/github/gparedesg/proyecto-integrador/blob/master/BERT%2BCFDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Entrenamiento de código de Productos y Servicios**

Para cada producto o servicio que queda en una factura timbrada en México tiene las mismas características. Debe llevar un código CFDI y una unidad CFDI asociada. Por ejemplo, un negocio que es un bar y vende alcohol puede tener en su menú tequila, mezcal, cerveza y ron. Puede venderlo en botellas, shots, jarras y vasos. Supongamos que hay un cliente que quiere comprar varios shots de mezcal. Si el cliente se llevó 3, entonces:

3 shot (unidad 14) de Mezcal (código 50202206)

Esta libreta busca entrenar el modelo de los códigos de producto y servicio únicamente, pues es la base de datos más extensa de las dos.

In [1]:
!pip install --upgrade tensorflow

In [2]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
df = pd.read_csv("/content/CFDI.csv")
print(df.head(5))

       code               description             similar       type  \
0   1010101  No existe en el catálogo  Público en general        NaN   
1  10101500  Animales vivos de granja                 NaN  Productos   
2  10101501               Gatos vivos                 NaN  Productos   
3  10101502                    Perros                 NaN  Productos   
4  10101504                     Visón                 NaN  Productos   

                                            division           group  
0                                                NaN             NaN  
1  Material Vivo Vegetal y Animal, Accesorios y S...  Animales vivos  
2  Material Vivo Vegetal y Animal, Accesorios y S...  Animales vivos  
3  Material Vivo Vegetal y Animal, Accesorios y S...  Animales vivos  
4  Material Vivo Vegetal y Animal, Accesorios y S...  Animales vivos  


En esta base de datos actualizada, tenemos además de la columna de frases similares, dos nuevas columnas para apoyarnos con el entrenamiento. Hay una división y un grupo, que se comportan como categoría y subcategoría dentro del catálogo CFDI.

In [4]:
df['group'].unique()

array([nan, 'Animales vivos', 'Productos para animales domésticos',
       'Comida de animales', 'Recipientes y hábitat para animales',
       'Productos de talabartería y arreo',
       'Semillas, bulbos, plántulas y esquejes',
       'Productos de floricultura y silvicultura',
       'Fertilizantes y nutrientes para plantas y herbicidas',
       'Productos para el control de plagas', 'Rosales vivos',
       'Plantas vivas de especies o variedades de flores altas',
       'Plantas vivas de especies o variedades de flores bajas',
       'Crisantemos vivos', 'Claveles vivos', 'Orquídeas vivas',
       'Rosas frescas cortadas',
       'Bouquets cortados frescos de especies o variedades de flores altas',
       'Bouquets cortados frescos de especies o variedades de flores bajas',
       'Crisantemos cortados frescos',
       'Bouquets florales cortados frescos', 'Claveles cortados frescos',
       'Orquídeas cortadas frescas', 'Rosas cortadas secas',
       'Bouquets cortados secos de esp

In [5]:
df['division'].unique()

array([nan, 'Material Vivo Vegetal y Animal, Accesorios y Suministros',
       'Material Mineral, Textil y Vegetal y Animal No Comestible',
       'Material Químico incluyendo Bioquímicos y Materiales de Gas',
       'Materiales de Resina, Colofonia, Caucho, Espuma, Película y Elastómericos',
       'Materiales y Productos de Papel',
       'Materiales Combustibles, Aditivos para Combustibles, Lubricantes y Anticorrosivos',
       'Maquinaria y Accesorios de Minería y Perforación de Pozos',
       'Maquinaria y Accesorios para Agricultura, Pesca, Silvicultura y Fauna',
       'Maquinaria y Accesorios para Construcción y Edificación',
       'Maquinaria y Accesorios para Manufactura y Procesamiento Industrial',
       'Maquinaria, Accesorios y Suministros para Manejo, Acondicionamiento y Almacenamiento de Materiales',
       'Vehículos Comerciales, Militares y Particulares, Accesorios y Componentes',
       'Maquinaria y Accesorios para Generación y Distribución de Energía',
       'Her

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52512 entries, 0 to 52511
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   code         52512 non-null  int64 
 1   description  52512 non-null  object
 2   similar      12201 non-null  object
 3   type         52511 non-null  object
 4   division     52510 non-null  object
 5   group        52511 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.4+ MB


Podemos observar que de los 52,512 productos o servicios, solamente 12,201 tienen una frase similar proporcionada por el SAT.

In [7]:
df[['description', 'group']] = df[['description', 'group']].replace('', np.nan)

df_filtered = df.dropna(subset=['description', 'group'])

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52511 entries, 1 to 52511
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   code         52511 non-null  int64 
 1   description  52511 non-null  object
 2   similar      12200 non-null  object
 3   type         52511 non-null  object
 4   division     52510 non-null  object
 5   group        52511 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.8+ MB


In [8]:
label_encoder = LabelEncoder()

df['group_encoded'] = label_encoder.fit_transform(df['group'])


In [9]:
encoding_table = pd.DataFrame({
    'Original': label_encoder.classes_,
    'Encoded': label_encoder.transform(label_encoder.classes_)
})

# Mostrar la tabla de codificación
print(encoding_table)

                               Original  Encoded
0    Accesorios de oficina y escritorio        0
1          Aceites y grasas comestibles        1
2                Adhesivos y selladores        2
3                              Aditivos        3
4                 Adornos para el hogar        4
..                                  ...      ...
403                     Vegetales secos      403
404                  Vehículos de motor      404
405                   Óptica industrial      405
406                      Óxido metálico      406
407                                 NaN      407

[408 rows x 2 columns]


In [10]:
#Comenzamos a crear el modelo

data_texts = df['description'].to_list()
data_labels = df['group_encoded'].to_list()

# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=0, shuffle=True)

# Keep some data for inference (testing)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.01, random_state=0, shuffle=True)

Se decidió usar 'distilbert-base-multilingual-cased' pero queremos buscar si existe un branch o alguna otra opción en Español

In [11]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((
dict(train_encodings),
train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
dict(val_encodings),
val_labels
))

In [13]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=408)

#optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(loss=model.hf_compute_loss, metrics=['accuracy'])

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
#early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(train_dataset.shuffle(1000).batch(100),
epochs=2,
batch_size=16,
validation_data=val_dataset.shuffle(1000).batch(100),
callbacks=[])

In [ ]:
# Procedemos a guardar el modelo
from tensorflow.keras.models import load_model
save_directory = "Multitext_Classification"

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

In [ ]:
# Para reusar el modelo lo cargamos

save_directory = "Multitext_Classification"
loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_directory)
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

In [ ]:
# Generamos una predicción



predict_input = loaded_tokenizer.encode("Una suscripción a una plataforma de apoyo legal",
truncation=True,
padding=True,
return_tensors="tf")

output = loaded_model(predict_input)[0]

prediction_value = tf.argmax(output, axis=1).numpy()[0]
print("Encoded Group:" + prediction_value)

In [ ]:
# Función para obtener el valor original dado un valor codificado
def get_original(encoded_value):
    result = encoding_table.loc[encoding_table['Encoded'] == encoded_value, 'Original']
    if not result.empty:
        return result.values[0]
    else:
        return None

original_value = get_original(prediction_value)
print(f'El valor original correspondiente a {encoded_value} es: {original_value}')